# Fair-Classification Preferential Fairness Example

Examples and explanations for function parameters https://github.com/mbilalzafar/fair-classification/tree/master/preferential_fairness.

In [1]:
import sys
import numpy as np
sys.path.append("../")

from aif360.datasets import GermanDataset
from aif360.metrics import ClassificationMetric
from fairensics.data.synthetic_dataset import SyntheticDataset
from fairensics.data.decision_boundary import DecisionBoundary 
from fairensics.methods import PreferentialFairness
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
# helper function to print evaluations
def print_evaluation(dataset, clf, unprivileged_groups, privileged_groups):
    predictions = clf.predict(dataset)

    metric = ClassificationMetric(dataset,
                                  predictions,
                                  unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups)

    print("p-rule: \t", metric.disparate_impact())
    print('accuracy: \t', metric.accuracy())
    

# 0.1 Load data

In [3]:
gcd = GermanDataset()

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

gcd_train, gcd_test = gcd.split([0.7], shuffle=True)

# 0.2 Metric
...

## 0.2.1 Definition 
- $z$ the protected features
- $X$ unprotected features
- $\hat{y}$ the predicted label
- $\hat{y}=1$ the positive label  (e.g. getting a loan)
- $P_i(\hat{y}=1)$ the probability for members of group _i_ to be assigned the positive label.


# 0.2.2 Method
...

# 0.3 Parameters
**- loss_function [string]**
- 'logreg' (default)
- 'svm_linear
- [logreg_2]

**- constraint_type [string]**
- None (default)
- "parity"
- "impact"
- "treatment"
- "both"

**- lam [number]**
- ...

**-  s_val_to_cons_sum**
 - ...

**- train_multiple [bool]**
 - if true, multiple classifiers are trained

**- Solver Parameters**
- tau
- mu
- EPS

In [4]:
pref = PreferentialFairness(warn=False)
print("PreferentialFairness without constraint")
pref.fit(gcd_train)
print_evaluation(gcd_test, pref, unprivileged_groups, privileged_groups)

PreferentialFairness without constraint
p-rule: 	 0.9507027625839508
accuracy: 	 0.7666666666666667


In [5]:
pref = PreferentialFairness(constraint_type="parity", warn=False)
print("PreferentialFairness with parity constraint")
pref.fit(gcd_train)
print_evaluation(gcd_test, pref, unprivileged_groups, privileged_groups)

PreferentialFairness with parity constraint
p-rule: 	 0.9712731836563938
accuracy: 	 0.76


In [6]:
pref = PreferentialFairness(constraint_type="treatment", train_multiple=True, warn=False)
print("PreferentialFairness with treatment constraint")
pref.fit(gcd_train)

print_evaluation(gcd_test, pref, unprivileged_groups, privileged_groups)

PreferentialFairness with treatment constraint
p-rule: 	 0.7588306127910087
accuracy: 	 0.7233333333333334


In [7]:
syn = SyntheticDataset()
lam = {0:2.5, 1:0.01}

pref = PreferentialFairness(constraint_type="impact",  lam=lam, train_multiple=True, warn=False)
print("PreferentialFairness with impact constraint")
pref.fit(syn)

print(pref._params["w"])

#print_evaluation(gcd_test, pref, unprivileged_groups, privileged_groups)

PreferentialFairness with impact constraint


../fairensics/methods/fairness_warnings.py:203: UserWarning: Ratio  between label and attribute  for protected attribute: protected_attribute is  above threshold of 0.4
  warnings.warn(warning_msg)


{0.0: array([None], dtype=object), 1.0: array([None], dtype=object)}


../fairensics/methods/preferential_fairness.py:244: UserWarning: Solver hasn't satisfied constraint. Make sure that the constraints are satisfied empirically. Alternatively, consider increasing tau parameter
  warnings.warn("Solver hasn't satisfied constraint."


In [8]:
pref = PreferentialFairness(constraint_type="both", lam=lam, train_multiple=True, warn=False)
print("PreferentialFairness with both constraints")
pref.fit(syn)

print(pref._params["w"][0])

#print_evaluation(gcd_test, pref, unprivileged_groups, privileged_groups)

../fairensics/methods/fairness_warnings.py:203: UserWarning: Ratio  between label and attribute  for protected attribute: protected_attribute is  above threshold of 0.4
  warnings.warn(warning_msg)


PreferentialFairness with both constraints
[None]


../fairensics/methods/preferential_fairness.py:244: UserWarning: Solver hasn't satisfied constraint. Make sure that the constraints are satisfied empirically. Alternatively, consider increasing tau parameter
  warnings.warn("Solver hasn't satisfied constraint."
